## Packages

In [1]:
import pandas as pd
import numpy as np

## Paths

In [2]:
DATA_PATH = './Data/pacientes_ucic_v3.csv'
OUTPUT_PATH = './Data/pacientes_ucic_xgb.csv'

## Read data

In [3]:
df = pd.read_csv(DATA_PATH)
df.head()

,Edad,Talla,Peso,SAPS-3,Tiempo de CEC,Tiempo de clampaje,Tiempo de isquemia,Tiempo de parada circulatoria,SEXO,cca_nyha_PF,...,Situación al alta de UCI.Ayuda movilización,Situación al alta de UCI.Estable no precisa cuidados especiales,Situación al alta de UCI.Precisa vigilancia herida quirúrgica,Situación al alta de UCI.Precisa telemetría,Situación al alta de UCI.Ayuda comer,Situación al alta de UCI.Vigilancia por delirio,Situación al alta de UCI.Vigilancia anemia/anticoagulantes,Situación al alta de UCI.Limitado esfuerzo terapéutico,Situación al alta de UCI.Cuidados especiales por vía aérea artificial,Situación al alta de UCI.Otro
0,83.095890,170.0,86.0,76.0,0.0,0.0,0.0,0.0,Hombre,III,...,0,0,0,0,0,0,0,0,0,0
1,82.084932,169.0,83.0,47.0,145.0,108.0,0.0,0.0,Hombre,II,...,0,0,0,0,0,0,0,0,0,0
2,51.216438,164.0,90.0,47.0,0.0,0.0,0.0,0.0,Hombre,III,...,1,0,0,0,0,0,0,0,0,0
3,78.284932,153.0,80.0,45.0,105.0,92.0,0.0,0.0,Mujer,III,...,0,1,0,0,0,0,0,0,0,0
4,63.887671,165.0,75.0,47.0,120.0,100.0,0.0,0.0,Hombre,III,...,0,0,0,0,0,0,0,0,0,0


## Selecting categorical columns

In [4]:
cat_cols = df.select_dtypes(include='object').columns
cat_cols

Index(['SEXO', 'cca_nyha_PF', 'Índice de Fragilidad',
       'cca_fevi_PF_varab_pre-ingr', 'Tipo de cirugía', 'Pico de troponina',
       'Sangrado en las primeras 24 horas', 'Cifra de plaquetas más baja',
       'núm. concentrados hematies primeras 48 h'],
      dtype='object')

## Function
It takes a pd.Series() and a list with ordered categories and return a factorized pd.Series()

In [5]:
def factorize_categorical(serie, ordered_classes):
    categorical_serie = pd.Categorical(serie.values, categories=ordered_classes)
    factorized_serie, _ = pd.factorize(categorical_serie, sort=True)
    return factorized_serie

## List
We define a list for each categorical attribute

In [6]:
sexo = [
    'Hombre',
    'Mujer'
]

cca_nyha_pf = [
    'I',
    'II',
    'III',
    'IV'
]

indice_fragilidad = [
    'Muy saludable. Por encima de lo esperado para su edad',
    'Sano: Paciente asintomático, con actividad normal',
    'Controlado: Paciente con sintomatología tratada, con activid',
    'Vulnerable:Independiente, pero con actividad limitada por su',
    'Fragilidad leve: Necesita ayuda para tareas difíciles',
    'Fragilidad moderada: Necesita ayuda dentro y fuera de la cas',
    'Fragilidad grave: Totalmente dependiente para las actividade',
    'Fragilidad muy grave: Totalmente dependiente, no tolera ni l',
    'Enfermedad terminal: Esperanza de vida menor de 6 meses'
]

cca_fevi_pf = [
    'No cuantificada',
    'Normal',
    'Disfunción leve (>40)',
    'Disfunción moderada (30-40)',
    'Disfunción grave (<30)'
]

tipo_cirugia = [
    'Programada',
    'Emergente (intervención en menos de 48 horas del diagnóstico',
    'Urgente (se interviene durante el ingreso de una descompensa'
]

pico_troponina = [
    'Menor de 500',
    '500-999',
    '1000-3999',
    '4000 o más'
]

sangrado_24 = [
    'Menor de 500 ml',
    '500-1000 ml',
    'Mayor de 1000 ml'
]

cifra_plaquetas = [
    '20.000-49.999',
    '50.000-99.999',
    '100.000-149.999',
    '>150.000'
]

num_hematies = [
    'Menor de 3',
    'Mayor o igual a 3'
]


## Dictionary
We define a dictionary with the lists

In [7]:
dict_ordered_classes = {
    'SEXO': sexo,
    'cca_nyha_PF': cca_nyha_pf,
    'Índice de Fragilidad': indice_fragilidad,
    'cca_fevi_PF_varab_pre-ingr': cca_fevi_pf,
    'Tipo de cirugía': tipo_cirugia,
    'Pico de troponina': pico_troponina,
    'Sangrado en las primeras 24 horas': sangrado_24,
    'Cifra de plaquetas más baja': cifra_plaquetas,
    'núm. concentrados hematies primeras 48 h': num_hematies
}

## We factorized every categorical column

In [8]:
for col in cat_cols:
    df[col] = factorize_categorical(df[col], dict_ordered_classes[col])
    df.loc[df[col] == -1, col] = np.nan

## Save new .csv

In [9]:
df.to_csv(OUTPUT_PATH, index=False)